In [1]:
import ee
import pandas as pd 
import numpy as np
import folium
from folium import plugins

### Calculating Evapotranspiration using Landsat 
1. We will calculate ET using the residual energy balance method 
- Net radiation (Rn)
- Soil Heat flux (G)
- Sensible Heat flux (H)

Then calculate $ LE=R_{n} - H - G $

### 1. Process Landsat 
- Select a Landsat Image
- Visualize 
- Get details 
- Cloud Mask 


In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
# Get a <10% cloudy landsat image of the central valley 
# LANDSAT/LC08/C02/T1_L2/LC08_043034_20220402
geometry=ee.Geometry.Point([-121.189405,37.615426])
# Get the image collection 
ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate("2022-04-01","2022-09-30").filterMetadata('CLOUD_COVER', 'less_than', 5).filterBounds(geometry);
print("The image collection has", ls.size().getInfo(), "images")
## Scale the images 
## Why do we need to scale? (Refer the slides)
def applyScaleFactors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
ls=ls.map(applyScaleFactors)
#Get the first image
ls_first=ls.first()

The image collection has 11 images


#### Visualize Image

In [4]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )}

In [6]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
# Set visualization parameters.
# vis_params = {
#   'min': 0,
#   'max': 4000,
#   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
vis_params = {
  'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
  'min': 0.0,
  'max': 0.3,
};
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_first, vis_params, 'False Color composite Image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

AttributeError: 'Map' object has no attribute 'add_ee_layer'

In [8]:
### Get Image details 
landsat_version=ls_first.get('L1_LANDSAT_PRODUCT_ID').getInfo()
print(landsat_version)
sun_elevation=ls_first.get("SUN_ELEVATION")
print("Sun Elevation ", sun_elevation.getInfo())
time_start=ls_first.get('system:time_start')
date=ee.Date(time_start)
year=ee.Number(date.get('year'))
month=ee.Number(date.get('month'))
day=ee.Number(date.get('day'))
hour=ee.Number(date.get('hour'))
minuts = ee.Number(date.get('minutes'))
print("Time of Image ", day.getInfo(),"/",month.getInfo(),"/",year.getInfo(), "@" , hour.getInfo(), ":",minuts.getInfo())

LC08_L1TP_043034_20220504_20220511_02_T1
Sun Elevation  62.39250282
Time of Image  4 / 5 / 2022 @ 18 : 39


In [9]:
date_string=date.format('YYYY-MM-dd').getInfo()

In [10]:
## Change Band Names (makes it easier to read)
print("Current Band Names: ",ls_first.bandNames().getInfo())
ls_first=ls_first.select([0,1,2,3,4,5,6,8,17], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","ST_B10","pixel_qa"])
print("New Band Names: ",ls_first.bandNames().getInfo())

Current Band Names:  ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
New Band Names:  ['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa']


#### Cloud Mask
Our image is <10% cloudy so we don't need to mask, but in general images can get cloudy. These cloudy pixels need to be removed (or made null) 

To do that, we take help of the "pixel_qa" band from Landsat 

In [11]:
# def f_cloudMaskL8_SR(image):
#     quality = image.select('pixel_qa');
#     ## Keep pixels than are clear and low confidence on clouds
#     c01 = quality.eq(21824); 
#     c02 = quality.eq(21952); 
#     c03=quality.eq(21888) 
#     c04 = quality.eq(21756);
#     mask = c01.Or(c02).Or(c03).Or(c04);
#     return image.updateMask(mask);
# ls_cloud=f_cloudMaskL8_SR(ls_first)

## 1. Net Radiation 
- Albedo
- Get ancilliary data
- Transmissivity
- $SW_{IN}$
- $LW_{OUT}$
- Cold pixel selection
- $LW_{IN}$ 

##### a) Albedo

In [12]:
def f_albedoL8(image):
    alfa = image.expression(
      '(0.130*B1) + (0.115*B2) + (0.143*B3) + (0.180*B4) + (0.281*B5) + (0.108*B6) + (0.042*B7)',{  #// (Ke, Im  et al 2016)
        'B1' : image.select(['UB']),
        'B2' : image.select(['B']),
        'B3' : image.select(['GR']),
        'B4' : image.select(['R']),
        'B5' : image.select(['NIR']),
        'B6' : image.select(['SWIR_1']),
        'B7' : image.select(['SWIR_2'])
      }).rename('ALFA');
    return image.addBands(alfa);
ls_first=f_albedoL8(ls_first)

In [13]:
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['ALFA'],
  'min': 0.0,
  'max': 0.3,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_first, vis_params, 'Albedo')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

### Q1. Looking at the image. Answer the following questions?
a) What does blue signify? What type of landcover is blue? 

b) What about the greens ? 

c) What are the red color and why? 

### Your answers here: a) Blue signifies low albedo. Water is typically assigned to blue due to its low albedo. b) Green is associated to vegetation. Green is the albedo level above water, so it might imply that the vegetation is well irrigated. c) Red is accociated with snow. Snow has a really high albedo, high albedo according to the color pallette is returned as red.

- get ancilliary data

In [16]:
import sys
sys.path.append('D:\\Backup\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\geeSEBAL_copy_edits\\etbrasil')
import geesebal
from geesebal import (tools,landsatcollection,masks,meteorology,endmembers, 
evapotranspiration,collection,timeseries,image,ET_Collection_mod)

In [17]:
### get Bounds of the landsat tile 
geometryReducer=ls_first.geometry().bounds().getInfo()
#  Get weather data from ERA-5
col_meteorology= meteorology.get_meteorology(ls_first,time_start);
print("Data contains: ",col_meteorology.bandNames().getInfo())
# Get Elevation data from SRTM 
SRTM_ELEVATION ='USGS/SRTMGL1_003'
srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
z_alt = srtm.select('elevation')

Data contains:  ['Rn24h_G', 'AirT_G', 'RH_G', 'ux_G', 'SW_Down']


In [18]:
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['AirT_G'],
  'min': 5,
  'max': 35,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(col_meteorology, vis_params, 'Air Temp (ERA-5)')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

In [23]:
ls_first.bandNames().getInfo()

['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']

In [31]:
#SPECTRAL INDICES MODULE
def fexp_spec_ind(image):
## Enter code for all the vegetation indices
    #NORMALIZED DIFFERENCE VEGETATION INDEX (NDVI)
    ndvi = image.normalizedDifference(['NIR', 'R']).rename('NDVI');

    #SOIL ADHUSTED VEGETATION INDEX (SAVI)
    savi = image.expression(
      '((1 + 0.5)*(B5 - B4)) / (0.5 + (B5 + B4))', {
        'B4': image.select('R'),
        'B5': image.select('NIR'),
    }).rename('SAVI');

    #NORMALIZED DIFFERENCE WATER INDEX (NDWI)
    ndwi = image.normalizedDifference(['GR', 'NIR']).rename('NDWI');
    
    savi1 = savi.where(savi.gt(0.689), 0.689);

    #LEAF AREA INDEX (LAI)
    lai = image.expression(
      '-(log(( 0.69-SAVI)/0.59 ) / 0.91)', {'SAVI': savi1}).rename('LAI');

    NDVI_adjust=(ndvi.clamp(0.0, 1.00))
    fipar = (NDVI_adjust.multiply(1).subtract(ee.Number(0.05))).rename('fipar')
    fipar = fipar.clamp(0,1)

    #BROAD-BAND SURFACE EMISSIVITY (e_0)
    e_0 = image.expression(
      '0.95 + 0.01 * LAI',{
        'LAI': lai});
    e_0 = e_0.where(lai.gt(3), 0.98).rename('e_0');

    #NARROW BAND TRANSMISSIVITY (e_NB)
    e_NB = image.expression(
      '0.97 + (0.0033 * LAI)',{'LAI': lai});
    e_NB = e_NB.where(lai.gt(3), 0.98).rename('e_NB');
    log_eNB = e_NB.log();

    #LAND SURFACE TEMPERATURE (LST) [K]
    lst=image.select("ST_B10").rename("T_LST")

    #FOR FUTHER USE
    pos_ndvi = ndvi.updateMask(ndvi.gt(0)).rename('pos_NDVI');
    ndvi_neg = pos_ndvi.multiply(-1).rename('NDVI_neg');
    lst_neg = lst.multiply(-1).rename('LST_neg');
    int_ = ee.Image(1).rename('int');
    lst_nw = lst.updateMask(ndwi.lte(0)).rename('LST_NW');
    sd_ndvi = ee.Image(1).rename('sd_ndvi');
    # ls_first=fexp_spec_ind(ls_first);

    #ADD BANDS
    image = image.addBands([ndvi,savi,lst,lai,e_0,e_NB,ndvi_neg,pos_ndvi,int_, sd_ndvi, ndwi,lst_neg,lst_nw]);
    return image
ls_first=fexp_spec_ind(ls_first)

In [54]:
### Transmissivity
def get_trans(image, z_alt, T_air, UR,SUN_ELEVATION,hour,minuts):
    #ATMOSPHERIC PRESSURE [KPA]
    #SHUTTLEWORTH (2012)
    pres = image.expression(
       '101.3 * ((293 - (0.0065 * Z))/ 293) ** 5.26 ', {
       'Z' : z_alt}).rename('P_ATM')

    #SATURATION VAPOR PRESSURE (es) [KPA]
    es = image.expression(
       ' 0.6108 *(exp( (17.27 * T_air) / (T_air + 237.3)))', {
       'T_air': T_air}).rename('ES')

    #ACTUAL VAPOR PRESSURE (ea) [KPA]
    ea = es.multiply(UR).divide(100).rename('EA')

    #WATER IN THE ATMOSPHERE [mm]
    #Garrison and Adler (1990)
    W = image.expression(
       '(0.14 * EA * PATM) + 2.1', {
       'PATM' : pres,
       'EA' : ea}).rename('W_ATM')

    #SOLAR ZENITH ANGLE OVER A HORZONTAL SURFACE
    solar_zenith = ee.Number(90).subtract(ee.Number(SUN_ELEVATION))
    degree2radian = 0.01745
    solar_zenith_radians = solar_zenith.multiply(ee.Number(degree2radian))
    cos_theta = solar_zenith_radians.cos()

    #BROAD-BAND ATMOSPHERIC TRANSMISSIVITY (tao_sw)
    #ASCE-EWRI (2005)
    ## -------------------------------------Enter code for emmissivity
    tao_sw = image.expression(
       '0.35 + 0.627 * exp(((-0.00146 * P)/(Kt * cos_theta)) - (0.075 * (W / cos_theta)**0.4))', {
       'P' : pres,
       'W': W,
       'Kt' : ee.Number(1),
       'cos_theta' : cos_theta}).rename('Tao_sw')
    
    #MASKS FOR SELECT PRE-CANDIDATES PIXELS
    lst_neg = image.select("ST_B10").multiply(-1).rename('LST_neg');
    ndwi = image.select('NDWI');

    lst_nw = image.select("ST_B10").updateMask(ndwi.lte(0)).rename('LST_NW');

    #ADD BANDS
    image=image.addBands([lst_neg, lst_nw,tao_sw,ea,W])

    return image
ls_trans=get_trans(ls_first,z_alt,col_meteorology.select("AirT_G"),col_meteorology.select("RH_G"),sun_elevation,hour,minuts)

In [55]:
ls_trans.bandNames().getInfo()

['UB',
 'B',
 'GR',
 'R',
 'NIR',
 'SWIR_1',
 'SWIR_2',
 'ST_B10',
 'pixel_qa',
 'ALFA',
 'NDVI',
 'SAVI',
 'T_LST',
 'LAI',
 'e_0',
 'e_NB',
 'NDVI_neg',
 'pos_NDVI',
 'int',
 'sd_ndvi',
 'NDWI',
 'LST_neg',
 'LST_NW',
 'LST_neg_1',
 'LST_NW_1',
 'Tao_sw',
 'EA',
 'W_ATM']

In [20]:
!pip install folium

In [21]:
!pip install geemap

In [56]:
# Added this myself from website
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
    
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['Tao_sw'],
  'min': 0.75,
  'max': 0.85,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}
vis_params_ndvi = {
  'bands': ['NDVI'],
  'min': -0.2,
  'max': 0.8,
   'palette': ['blue','red', 'orange',"yellow",'green']}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_trans, vis_params, 'Transmissivity')
my_map.add_ee_layer(ls_first, vis_params_ndvi, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

In [57]:
#INSTANTANEOUS INCOMING SHORT-WAVE RADIATION (Rs_down) [W M-2]
def fexp_radshort_down(image, z_alt, T_air, UR,SUN_ELEVATION):
    ## Fill the required gaps (Rename Band to Rs_down) 
    #SOLAR CONSTANT
    gsc = ee.Number(1367) #[W M-2]

    #DAY OF THE YEAR
    dateStr = image.date()
    doy = dateStr.getRelative('day', 'year')
    Pi=ee.Number(3.14)

    #INVERSE RELATIVE  DISTANCE EARTH-SUN
    d1 = ee.Number(2).multiply(Pi).divide(ee.Number(365));
    d2 = d1.multiply(doy);
    d3 = d2.cos();
    dr = ee.Number(1).add(ee.Number(0.033).multiply(d3));

    #ATMOSPHERIC PRESSURE [KPA]
    #SHUTTLEWORTH (2012)
    pres = image.expression(
       '101.3 * ((293 - (0.0065 * Z))/ 293) ** 5.26 ', {
       'Z' : z_alt}).rename('P_ATM')

    #SATURATION VAPOR PRESSURE (es) [KPA]
    es = image.expression(
       ' 0.6108 *(exp( (17.27 * T_air) / (T_air + 237.3)))', {
       'T_air': T_air}).rename('ES')

    #ACTUAL VAPOR PRESSURE (ea) [KPA]
    ea = es.multiply(UR).divide(100).rename('EA')

    #WATER IN THE ATMOSPHERE [mm]
    #GARRISON AND ADLER (1990)
    W = image.expression(
       '(0.14 * EA * PATM) + 2.1', {
       'PATM' : pres,
       'EA' : ea, }).rename('W_ATM')

    #SOLAR ZENITH ANGLE OVER A HORIZONTAL SURFACE
    solar_zenith = ee.Number(90).subtract(ee.Number(SUN_ELEVATION))
    degree2radian = 0.01745
    solar_zenith_radians = solar_zenith.multiply(ee.Number(degree2radian))
    cos_theta = solar_zenith_radians.cos()

    #BROAD-BAND ATMOSPHERIC TRANSMISSIVITY (tao_sw)
    #ASCE-EWRI (2005)
    tao_sw = image.expression(
       '0.35 + 0.627 * exp(((-0.00146 * P)/(Kt * cos_theta)) - (0.075 * (W / cos_theta)**0.4))', {
       'P' : pres,
       'W': W,
       'Kt' : ee.Number(1),
       'cos_theta' : cos_theta}).rename('Tao_sw')

    #INSTANTANEOUS SHORT-WAVE RADIATION (Rs_down) [W M-2]
    Rs_down = image.expression(
       '(gsc * cos_theta * tao_sw * dr)', {
       'gsc' : gsc,
       'tao_sw': tao_sw,
       'dr' : dr,
       'cos_theta' : cos_theta}).rename('Rs_down')

    #ADD BANDS
    image =  image.addBands([Rs_down, tao_sw,es,ea]);
    return image
ls_trans=fexp_radshort_down(ls_trans,z_alt,col_meteorology.select("AirT_G"),col_meteorology.select("RH_G"),sun_elevation)

In [58]:
#INSTANTANEOUS OUTGOING LONG-WAVE RADIATION (Rl_up) [W M-2]
def fexp_radlong_up(image):
 
    ## Enter code (Rename band "Rl_up")

   emi = image.expression('0.95 + (0.01 * LAI)', {
           'LAI' : image.select('LAI')})
   #LAI
   lai = image.select('LAI');
   emi = emi.where(lai.gt(3), 0.98)
   stefBol = ee.Image(5.67e-8)

   Rl_up = image.expression(
      'emi * stefBol * (LST ** 4)', {
      'emi' : emi,
      'stefBol': stefBol,
      'LST': image.select('T_LST')}).rename('Rl_up')

   #ADD BANDS
   image = image.addBands([Rl_up])
   return image
ls_trans=fexp_radlong_up(ls_trans)


In [59]:
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['Rl_up'],
  'min': 400,
  'max': 700,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}

# vis_params_ndvi = {
#   'bands': ['NDVI'],
#   'min': -0.2,
#   'max': 0.8,
#    'palette': ['blue','red', 'orange',"yellow",'green']}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_trans, vis_params, 'LW_OUT')
# my_map.add_ee_layer(ls_trans, vis_params_sw, 'SW_IN')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

### Incoming Longwave Radiation

In [60]:
## We need to select a cold pixel
NDVI_cold=5
Ts_cold=20
NDVI_hot=10
Ts_hot=20
p_top_NDVI=ee.Number(NDVI_cold)
p_coldest_Ts=ee.Number(Ts_cold)
p_lowest_NDVI=ee.Number(NDVI_hot)
p_hottest_Ts=ee.Number(Ts_hot)
d_cold_pixel=endmembers.fexp_cold_pixel(ls_trans, geometryReducer, p_top_NDVI, p_coldest_Ts)
d_cold_pixel.getInfo()

{'ndvi': 0.7832890152931213,
 'sum': 525382,
 'temp': 295.06224866,
 'x': None,
 'y': None}

In [61]:
def fexp_radlong_down(image, n_Ts_cold):
    ## Enter code (Rename band "Rl_down")
    log_taosw = image.select('Tao_sw').log()
    Rl_down = image.expression(
       '(0.85 * (- log_taosw) ** 0.09) * stefBol * (n_Ts_cold ** 4)  ',{
       'log_taosw' : log_taosw,
       'stefBol': ee.Number(5.67e-8),
       'n_Ts_cold' : n_Ts_cold}).rename('Rl_down')

    #ADD BANDS
    image = image.addBands([Rl_down])
    return image
ls_trans=fexp_radlong_down(ls_trans,ee.Number(d_cold_pixel.get("temp").getInfo()))

In [63]:
ls_trans.bandNames().getInfo()

['UB',
 'B',
 'GR',
 'R',
 'NIR',
 'SWIR_1',
 'SWIR_2',
 'ST_B10',
 'pixel_qa',
 'ALFA',
 'NDVI',
 'SAVI',
 'T_LST',
 'LAI',
 'e_0',
 'e_NB',
 'NDVI_neg',
 'pos_NDVI',
 'int',
 'sd_ndvi',
 'NDWI',
 'LST_neg',
 'LST_NW',
 'LST_neg_1',
 'LST_NW_1',
 'Tao_sw',
 'EA',
 'W_ATM',
 'Rs_down',
 'Tao_sw_1',
 'ES',
 'EA_1',
 'Rl_up',
 'Rl_down']

In [64]:
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['Rl_down'],
  'min': 310,
  'max': 325,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}
# vis_params_ndvi = {
#   'bands': ['NDVI'],
#   'min': -0.2,
#   'max': 0.8,
#    'palette': ['blue','red', 'orange',"yellow",'green']}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_trans, vis_params, 'LW_IN')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

#### Net radiation

In [65]:
def fexp_radbalance(image):
    ## --- Enter code (Rename Band Rn)
    Rn = image.expression(
       '((1-alfa) * Rs_down) + Rl_down - Rl_up - ((1 - e_0) * Rl_down) ', {
       'alfa' : image.select('ALFA'),
       'Rs_down': image.select('Rs_down'),
       'Rl_down' : image.select('Rl_down'),
       'Rl_up' : image.select('Rl_up'),
       'e_0' : image.select('e_0')}).rename('Rn')
    #ADD BANDS
    image = image.addBands(Rn)
    return image
ls_trans=fexp_radbalance(ls_trans)

In [66]:
ls_trans.bandNames().getInfo()

['UB',
 'B',
 'GR',
 'R',
 'NIR',
 'SWIR_1',
 'SWIR_2',
 'ST_B10',
 'pixel_qa',
 'ALFA',
 'NDVI',
 'SAVI',
 'T_LST',
 'LAI',
 'e_0',
 'e_NB',
 'NDVI_neg',
 'pos_NDVI',
 'int',
 'sd_ndvi',
 'NDWI',
 'LST_neg',
 'LST_NW',
 'LST_neg_1',
 'LST_NW_1',
 'Tao_sw',
 'EA',
 'W_ATM',
 'Rs_down',
 'Tao_sw_1',
 'ES',
 'EA_1',
 'Rl_up',
 'Rl_down',
 'Rn']

In [67]:
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['Rn'],
  'min': 350,
  'max': 900,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}
# vis_params_ndvi = {
#   'bands': ['NDVI'],
#   'min': -0.2,
#   'max': 0.8,
#    'palette': ['blue','red', 'orange',"yellow",'green']}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_trans, vis_params, 'Net Radiation')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

Q1. Which land cover shows highest net radiation and which shows the lowest? 

Answer Here

Q2. Why is it high and low in those landcover? (Think which component contributes most to net radiation)

Answer here

### Soil Heat Flux

In [68]:
#SOIL HEAT FLUX (G) [W M-2]
#BASTIAANSSEN (2000)
def fexp_soil_heat(image):
    ## Enter Code (reName band "G")
    G = image.expression(
        'Rn * (T - 273.15) * ( 0.0038 + (0.0074 * ALFA)) *  (1 - 0.98 * (NDVI ** 4)) ', {
         'Rn' : image.select('Rn'),
         'NDVI': image.select('NDVI'),
         'ALFA': image.select('ALFA'),
         'T': image.select('ST_B10')}).rename('G');
    #ADD BANDS
    image = image.addBands(G);
    return image
ls_trans=fexp_soil_heat(ls_trans)

In [69]:
ls_trans.bandNames().getInfo()

['UB',
 'B',
 'GR',
 'R',
 'NIR',
 'SWIR_1',
 'SWIR_2',
 'ST_B10',
 'pixel_qa',
 'ALFA',
 'NDVI',
 'SAVI',
 'T_LST',
 'LAI',
 'e_0',
 'e_NB',
 'NDVI_neg',
 'pos_NDVI',
 'int',
 'sd_ndvi',
 'NDWI',
 'LST_neg',
 'LST_NW',
 'LST_neg_1',
 'LST_NW_1',
 'Tao_sw',
 'EA',
 'W_ATM',
 'Rs_down',
 'Tao_sw_1',
 'ES',
 'EA_1',
 'Rl_up',
 'Rl_down',
 'Rn',
 'G']

In [70]:
# Set visualization parameters.
folium.Map.add_ee_layer = add_ee_layer
vis_params = {
  'bands': ['G'],
  'min': 40,
  'max': 140,
   'palette': ['blue', 'green', 'yellow', 'orange',"red"]}
# vis_params_ndvi = {
#   'bands': ['NDVI'],
#   'min': -0.2,
#   'max': 0.8,
#    'palette': ['blue','red', 'orange',"yellow",'green']}
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=4, height=500)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
# basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(ls_trans, vis_params, 'Soil Heat')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

Q4. What influences soil heat flux? Can you find a logical explanation of what you see? 

Answer here

### Sensible Heat Flux 

In [72]:
##HOT PIXEL
d_hot_pixel=endmembers.fexp_hot_pixel(ls_trans, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
# ##SENSIBLE HEAT FLUX (H) [W M-2]
ls_trans=tools.fexp_sensible_heat_flux(ls_trans, col_meteorology.select("ux_G"), col_meteorology.select("RH_G"),col_meteorology.select("Rn24h_G"),ee.Number(d_cold_pixel.get("temp").getInfo()) \
                                       ,d_hot_pixel,date_string,geometryReducer)

EEException: Invalid argument specified for ee.Number(): None

### Final Task 
Calculate Latent Heat Flux (LE) as residual energy Balance and plot it

### Deliverables

- Plots 
    - All components of Radiation
    - H, LE and G fluxes 
- This Jupyter Notebook(.ipnyb)
    